In [1]:
# importing the necessary libraries
import glob
import os
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
import openpyxl

In [2]:
# defining the paths
shapefile_path = "/shared_space/BrazilSPEI/Maranhao_boundaries_EPSG/maranhao_boundary.shp"
data_dir = "/shared_space/BrazilSPEI/MODIS/Reclass_Data"
output_gif = "maranhao_multiclass_animation.gif"

fps = 2
dpi = 150
interval = 600

In [3]:
# loading the Maranhao file
print("Loading Maranhão shapefile...")
ma = gpd.read_file(shapefile_path)
ma.crs = "EPSG:4326"

Loading Maranhão shapefile...


In [4]:
# loading the multiclass rasters
print("Loading multiclass rasters...")
multiclass_paths = sorted(glob.glob(os.path.join(data_dir, "*_multiclass.tif")))

print(f"Found {len(multiclass_paths)} multiclass rasters")

Loading multiclass rasters...
Found 24 multiclass rasters


In [5]:
# extracting the years
years = []
for path in multiclass_paths:
    fname = os.path.basename(path)
    digits = ''.join(filter(str.isdigit, fname))
    years.append(int(digits[-4:]) if len(digits) >= 4 else None)

In [10]:
rows = []

for path, year in zip(multiclass_paths, years):
    with rasterio.open(path) as src:
        arr = src.read(1).astype(float)

    forest_area = np.sum(arr == 1) * 0.25
    agri_area = np.sum(arr == 2) * 0.25
    pasture_area = np.sum(arr == 3) * 0.25

    rows.append({
        "Year": year,
        "Forest (km²)": round(forest_area),
        "Agriculture (km²)": round(agri_area),
        "Pasture (km²)": round(pasture_area)
    })

df = pd.DataFrame(rows)

print(df.to_string(index=False))

df.to_csv("deforestation_table.csv", index=False)
df.to_excel("deforestation_table.xlsx", index=False)


Agriculture (km²)  Forest (km²)  Pasture (km²)  Year
           2605.0       60612.0       231134.0  2001
           2472.0       54437.0       239392.0  2002
           2642.0       49906.0       244002.0  2003
           2764.0       45960.0       246360.0  2004
           2976.0       44393.0       247262.0  2005
           3203.0       40937.0       250018.0  2006
           3357.0       39440.0       250238.0  2007
           3726.0       34968.0       252082.0  2008
           3899.0       34334.0       254068.0  2009
           3956.0       34740.0       252582.0  2010
           4225.0       36248.0       251115.0  2011
           4402.0       35394.0       251080.0  2012
           4539.0       32732.0       253936.0  2013
           4611.0       32719.0       254272.0  2014
           4853.0       33545.0       250632.0  2015
           4524.0       30428.0       247305.0  2016
           4874.0       32213.0       243316.0  2017
           5334.0       32915.0       239701.0